In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
%%time

df=pd.read_csv("en.openfoodfacts.org.products.csv",
              sep="\t", encoding="utf-8", low_memory=False)

CPU times: total: 7min 42s
Wall time: 15min 5s


In [3]:
%%time

df

CPU times: total: 0 ns
Wall time: 0 ns


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,product_name,abbreviated_product_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,nutrinet-sante,jeunes pousses,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1656948610,2022-07-04T15:30:10Z,1656948613,2022-07-04T15:30:13Z,kiliweb,Andrè,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1656488106,2022-06-29T07:35:06Z,stephane,L.casei,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1654250311,2022-06-03T09:58:31Z,1654270474,2022-06-03T15:34:34Z,quentinbrd,Skyr,NaN,...,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,kiliweb,Vitória crackers,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814747,9999999990397,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1581171613,2020-02-08T14:20:13Z,1582644247,2020-02-25T15:24:07Z,fix-salt-bot,Fati,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2814748,9999999999,http://world-en.openfoodfacts.org/product/9999...,prepperapp,1666599417,2022-10-24T08:16:57Z,1666599418,2022-10-24T08:16:58Z,prepperapp,Duschgel - Iris,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2814749,999999999999,http://world-en.openfoodfacts.org/product/9999...,openfoodfacts-contributors,1674510561,2023-01-23T21:49:21Z,1674510561,2023-01-23T21:49:21Z,NaN,Test Product Product,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2814750,9999999999994,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1613129728,2021-02-12T11:35:28Z,1660330421,2022-08-12T18:53:41Z,franck-pepper,Light & Free SKYR A BOIRE,NaN,...,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 0.Filtrer le dataframe

In [ ]:
from t0_datafilter import datas_filter

In [ ]:
filter = datas_filter()

filter.downcast(df, show_saved_memory=True)

# filter exemples
print("Mise en place des filtres :")
print("Colones numériques : ")
print(list(filter.filter(df, type="number", nan_percent=34).columns))
print("Colones ordinales : ")
#print(list(filter.filter(df, type="ordinal", ordinal_features_names=["i"]).columns))
print("Colones non-ordinales (max 2): ")
print(list(filter.filter(df, type="non-ordinal", category_count=2).columns))
print("Colones catégorielles : ")
print(list(filter.filter(df, type="categorical").columns))

## 1.0.Nettoyer le jeu de données des valeurs problématiques

In [ ]:
def count_duplicated_values(df):
        """Count duplicated values for each feature
        Returns:
            df (DataFrame): return sum of duplicated values
        @Author: Thomas PAYAN
        """
        return df.duplicated().sum()

In [ ]:
def count_duplicate_rows(df):
    """Compte le nombre de lignes en doublon dans un DataFrame"""
    duplicates = df.duplicated()
    return duplicates.sum()

count_duplicate_rows(df)

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing.sort_values(ascending=False)

threshold_view = 2

ax = sns.barplot(x = percent_missing[percent_missing.values > threshold_view], 
                 y=percent_missing[percent_missing.values > threshold_view].index, 
                 orient='h');
ax.set_title(f"Répartition du pourcentage de valeurs manquantes supérieures au seuil de 2%")

In [ ]:
def drop_duplicated_values(df):
        """Drop duplicated values
        Returns:
            df (DataFrame): return dataframe without duplicated values
        @Author: Thomas PAYAN
        """
        print("\nRows number before drop duplicated values : %s" % len(df))
        if(df.count_duplicated_values() > 0):
            df.drop_duplicates(inplace=True)
        print("\nRows number after drop duplicated values : %s" % len(df))
        return df

In [ ]:
from t1_0_cleandata import drop_columns_with_missing_values, impute_missing_values

In [ ]:
df = drop_columns_with_missing_values(df, threshold=0.95)

In [ ]:
df.dtypes

In [ ]:
df.select_dtypes(include=np.number).columns.tolist()

In [ ]:
columns = ['created_t',
 'last_modified_t',
 'serving_quantity',
 'additives_n',
 'nutriscore_score',
 'nova_group',
 'ecoscore_score',
 'product_quantity',
 'unique_scans_n',
 'completeness',
 'last_image_t',
 'energy-kcal_100g',
 'energy_100g',
 'fat_100g',
 'saturated-fat_100g',
 'trans-fat_100g',
 'cholesterol_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'proteins_100g',
 'salt_100g',
 'sodium_100g',
 'vitamin-a_100g',
 'vitamin-c_100g',
 'calcium_100g',
 'iron_100g',
 'fruits-vegetables-nuts-estimate-from-ingredients_100g',
 'nutrition-score-fr_100g']

In [ ]:
impute_missing_values(df, columns, missing_values='NaN', n_neighbors=5, weights='uniform')